The purpose of this notebook is to loop through the rows of a dataset and and calculate the amount of time that the ferry was delayed for each row. Then a new column is created with this data. This loop below does this for all `csv` files in `"../csv-files/Step 1 - Indexed/"` 

In [1]:
def minutesToSeconds(minutes):
    if(":" in minutes):
        m, s = minutes.split(':')
        return (int(m)*60)+int(float(s))
    return (int(minutes)*60)

def pastHourDepartureDifference(actual_departure_minutes, scheduled_departure_minutes):
    # Calculate how far from end of the hour schedule_departure_minutes is
    actual_departure_past_hour = (60*60) - scheduled_departure_minutes
    
    # Add this on to the actual departure minutes
    return actual_departure_past_hour + actual_departure_minutes

In [2]:
def calculate_seconds_late(df):
    seconds_late = []

    for index, row in df.iterrows():

        if(not str(row.iloc[4]) == "nan"):
            actual_departure_seconds = minutesToSeconds(row.iloc[4])
        else:
            seconds_late.append(float('nan'))
            continue

        scheduled_departure_seconds = minutesToSeconds((row.iloc[3].split(":")[0]))

        # ex: actual_departure (05:56.0), scheduled_departure (0:55:00)
        if(actual_departure_seconds < scheduled_departure_seconds):
            difference = abs(actual_departure_seconds - scheduled_departure_seconds)

            # Making the assumption here that more than 5 minutes early is not possible.
            if(difference > (5*60)):
                seconds_late.append(pastHourDepartureDifference(actual_departure_seconds, scheduled_departure_seconds))
            #If less than 5 minutes then lets take it
            else:
                seconds_late.append((actual_departure_seconds - scheduled_departure_seconds))      
        elif(actual_departure_seconds > scheduled_departure_seconds):
            seconds_late.append(actual_departure_seconds - scheduled_departure_seconds)
        elif(actual_departure_seconds == scheduled_departure_seconds):
            seconds_late.append(0)
            
    return seconds_late

In [3]:
import os
import pandas as pd

for filename in os.listdir("../csv-files/Step 1 - Indexed/"):
    df = pd.read_csv("../csv-files/Step 1 - Indexed/%s" % (filename), parse_dates = [0])
    seconds_late = calculate_seconds_late(df)
    df["seconds_late"] = seconds_late
    df.to_csv("../csv-files/Step 2 - Seconds Late/Bainbridge_Departures_%s_with_Seconds_Late.csv" % filename.split("_")[2],
             index = False)